In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image

In [2]:
scyjava.config.add_option('-Xmx30g')
start_dir = os.getcwd()
#ij = imagej.init('sc.fiji:fiji', mode='interactive')
ij = imagej.init('/home/saka/sw/local/fiji/2023', mode='interactive')
## Something about this init() function changes the current working directory.
ij.getApp().getInfo(True)
ij.ui().showUI()
os.chdir(start_dir)
ij.getVersion() #This is to make sure ImageJ/Fiji opened properly. In case of error, it could display '2.9.0/inactive' instead of the full version number

'2.9.0/1.53t'

121


In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
ov = Overlay()

In [4]:
Overlay = scyjava.jimport('ij.gui.Overlay')
ov = Overlay()
rm = ij.RoiManager.getRoiManager()

In [5]:
root = tk.Tk()
root.withdraw()
filepath = filedialog.askopenfilename()


In [6]:
directory_path = filedialog.askdirectory()

In [7]:
image = ij.io().open(filepath)
ij.ui().show(image)
imp = ij.py.to_imageplus(image)

In [8]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
num_times = data_info['T']


In [9]:
print(data_info['T'])

In [10]:
for i in range (data_info['T']):
    macro = f"Stack.setFrame({i + 1})"
    ij.py.run_macro(macro)
    input_txt = Path(f"{directory_path}/frame_{i}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
    rm.runCommand("Select", f"{i}")
    rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
rm.runCommand("Save", f"{folder_path}/" + f"{f_name}.zip")

NameError: name 'folder_path' is not defined